In [64]:
import numpy as np 
import pandas as pd  
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, DecisionTreeRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb





ImportError: cannot import name 'DecisionTreeRegressor' from 'sklearn.linear_model' (c:\Users\sk531\anaconda3\lib\site-packages\sklearn\linear_model\__init__.py)

In [ ]:
df=pd.read_csv("gurgaon_properties_post_feature_selection_v2.csv")

In [ ]:
df.head()

 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, DecisionTreeRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb


,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,0.0,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,0.0,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,0.0,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,1.0,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,0.0,High,Mid Floor


In [ ]:
df['furnishing_type'].value_counts()

0.0    2349
1.0    1018
2.0     187
Name: furnishing_type, dtype: int64

In [ ]:
# 0 ->unfurnished
# 1-> semifurnished 
# 2-> furnished 

df['furnishing_type']=df['furnishing_type'].replace({
    0.0:'unfurnished',
    1.0:'semifurnished',
    2.0:"furnished"
})

In [ ]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,unfurnished,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,unfurnished,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,semifurnished,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,unfurnished,High,Mid Floor


In [ ]:
X=df.drop(columns=['price'])
y=df['price']

In [ ]:
y

0        0.82
1        0.95
2        0.32
3        1.60
4        0.48
        ...  
3549     0.37
3550     6.00
3551     0.60
3552    15.50
3553     1.78
Name: price, Length: 3554, dtype: float64

In [ ]:
# Applying the log1p transformation to the target variable 

y_transformed=np.log1p(y)
print(y)
print(y_transformed)

0        0.82
1        0.95
2        0.32
3        1.60
4        0.48
        ...  
3549     0.37
3550     6.00
3551     0.60
3552    15.50
3553     1.78
Name: price, Length: 3554, dtype: float64
0       0.598837
1       0.667829
2       0.277632
3       0.955511
4       0.392042
          ...   
3549    0.314811
3550    1.945910
3551    0.470004
3552    2.803360
3553    1.022451
Name: price, Length: 3554, dtype: float64


### Ordinal Encoding 

In [ ]:
print(df.columns)
df.head()

Index(['property_type', 'sector', 'price', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')


,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,unfurnished,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,unfurnished,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,semifurnished,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,unfurnished,High,Mid Floor


In [ ]:
columns_to_encode=[
    'property_type',
    'sector',
    'balcony',
    'agePossession',
    'furnishing_type',
    'luxury_category',
    'floor_category'
]

In [ ]:
# creating a column tranformer for preprocessing 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [ ]:
# Creating a pipeline 
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [ ]:
# K-fold cross-validation 
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')


In [ ]:
scores.mean(),scores.std()

(0.7363096633436826, 0.03238005754429935)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [ ]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [ ]:
y_pred=pipeline.predict(X_test)


In [ ]:
y_pred=np.expm1(y_pred)

In [ ]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.9463822160089357

In [ ]:
def scorer(model_name,model):
    output=[]
    output.append(model_name)
    pipeline=Pipeline([
        ('preprocessor',preprocessor),
        ('regressor',model)
    ])

    # K-fold cross validation 
    kflod=KFold(n_splits=10,shuffle=True,random_state=42)
    scores=cross_val_score(pipeline,X,y_transformed,cv=kflod,scoring='r2')

    output.append(scores.mean())

    X_train,X_test,y_train,y_test=train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)
    y_pred=pipeline.predict(X_test)
    y_pred=np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    return output

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

model_dict = {
    'linear_reg': LinearRegression(),
    'svr': SVR(),
    'ridge': Ridge(),
    'LASSO': Lasso(),
    'decision_tree': DecisionTreeRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'random_forest': RandomForestRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost': xgb.XGBRegressor()
}


In [ ]:
model_output=[]
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name,model))

In [ ]:
model_output

[['linear_reg', 0.7363096633436826, 0.9463822160089357],
 ['svr', 0.7642012011196353, 0.8472636473483934],
 ['ridge', 0.7363125343993555, 0.9463387741853383],
 ['LASSO', 0.05943378064493573, 1.528905986892753],
 ['decision_tree', 0.7680430457634635, 0.7628481229975675],
 ['extra_trees', 0.868345376170927, 0.5428696008114978],
 ['random_forest', 0.8800826070670078, 0.5277242304250368],
 ['gradient_boosting', 0.8726004282056732, 0.5761049500775237],
 ['adaboost', 0.7583832228650257, 0.8245719129500592],
 ['mlp', 0.8125696803572536, 0.6970375579538141],
 ['xgboost', 0.8894876835260124, 0.5040475141482346]]

In [ ]:
model_df=pd.DataFrame(model_output,columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.889488,0.504048
6,random_forest,0.880083,0.527724
5,extra_trees,0.868345,0.542870
7,gradient_boosting,0.872600,0.576105
9,mlp,0.812570,0.697038
4,decision_tree,0.768043,0.762848
8,adaboost,0.758383,0.824572
1,svr,0.764201,0.847264
2,ridge,0.736313,0.946339
0,linear_reg,0.736310,0.946382


### OneHotEncoding With PCA


In [ ]:
print(df.columns )
df.head()


# pca applied for dimensionality reduction so in the pipeline after OHE apply PCA for dimensionality reduction 

Index(['property_type', 'sector', 'price', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')


,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,unfurnished,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,unfurnished,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,semifurnished,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,unfurnished,High,Mid Floor


In [ ]:
# creating a column trannsformer for preprocessing 
preprocessor=ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),['bedRoom','bathroom','built_up_area','servant room','store room'])
        ,
        ('cat',OrdinalEncoder(),columns_to_encode)
        ,
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ],
    remainder='passthrough'
)
# For example, if you have a categorical feature "Color" with categories "Red", "Blue", and "Green", and you encode it using one-hot encoding with drop='first', you'll get two binary features: "Is Blue" and "Is Green". If both of these features are 0, then it implies that the color is "Red".

In [ ]:
# creating a pipeline 
pipeline=Pipeline([
    ('preprocessor',preprocessor),
    ('pca',PCA(n_components=0.95)),
    ('regressor',LinearRegression())
])

In [ ]:
# K-fold cross validation 
kfold=KFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_val_score(pipeline,X,y_transformed,cv=kfold,scoring='r2')


In [ ]:
scores.mean()

0.06225201431451133

In [ ]:
scores.std()

0.019860594071640147

In [ ]:
def scorer(model_name,model):
    output=[]
    output.append(model_name)
    pipeline=Pipeline([
        ('preprocessor',preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('regressor',model)
    ])

    # K-fold cross validation 
    kflod=KFold(n_splits=10,shuffle=True,random_state=42)
    scores=cross_val_score(pipeline,X,y_transformed,cv=kflod,scoring='r2')

    output.append(scores.mean())

    X_train,X_test,y_train,y_test=train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)
    y_pred=pipeline.predict(X_test)
    y_pred=np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    return output

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

model_dict = {
    'linear_reg': LinearRegression(),
    'svr': SVR(),
    'ridge': Ridge(),
    'LASSO': Lasso(),
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost': xgb.XGBRegressor()
}


In [ ]:
model_output=[]
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name,model))

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random_forest,0.761721,0.658519
6,extra_trees,0.739724,0.700945
4,decision_tree,0.696182,0.757290
10,xgboost,0.620664,0.948597
7,gradient_boosting,0.610604,0.987906
8,adaboost,0.315991,1.356678
1,svr,0.218073,1.361163
9,mlp,0.213885,1.420333
2,ridge,0.062252,1.526707
0,linear_reg,0.062252,1.526707


### Target Encoder 

is used for those columns in which high cardinality is there /dimesionality jese sector 

here we create a column for sector and than suppose we go to all sector 1 and find the mean and replace with mean in place of sector 
category ko ouptut coloumn ke basis pe decide karte hai and so is the name target encoding 
suppose in hotel we have preception that italian food is costliter than north indian based on hotels we went and checked the price so for italian we have some price range fixed in our mind that may be suppose 600 and for north indian its 200 


In [ ]:
df.columns

Index(['property_type', 'sector', 'price', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [ ]:
# pip install category_encoders

     -------------------------------------- 81.9/81.9 kB 328.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df.head(1)

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor


In [ ]:
import category_encoders as ce 

columns_to_encode=['property_type','sector','balcony','agePossession','furnishing_type','luxury_category','floor_category']

# creating a column tranformer for preprocessing


preprocessor=ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),['bedRoom','bathroom','built_up_area','servant room','store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc',ce.TargetEncoder(),['sector'])
    ],
    remainder='passthrough'
)


In [ ]:
pipeline=Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',LinearRegression())
])

In [ ]:
# Kfold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')


In [ ]:
scores.mean(),scores.std()

(0.829521918225536, 0.018384463379122827)

In [ ]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output  

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

model_dict = {
    'linear_reg': LinearRegression(),
    'svr': SVR(),
    'ridge': Ridge(),
    'LASSO': Lasso(),
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost': xgb.XGBRegressor()
}


In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [65]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [66]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random_forest,0.900553,0.445143
10,xgboost,0.904798,0.447518
6,extra_trees,0.902125,0.459941
7,gradient_boosting,0.889011,0.507558
4,decision_tree,0.828527,0.580674
9,mlp,0.852504,0.637040
8,adaboost,0.816011,0.702870
0,linear_reg,0.829522,0.713011
2,ridge,0.829536,0.713523
1,svr,0.782917,0.818851


So conclusion from this is that 
-> Tree Based algorithms ->rf/xg is giving good results 
-> Use target encoding on sector 

### Hyperparameter Tuning 

In [68]:
from sklearn.model_selection import GridSearchCV



In [69]:
param_grid={
    'regressor__n_estimators':[50,100,200,300],
    'regressor__max_depth':[None,10,20,30],
    'regressor__max_samples':[0.1,0.25,0.5,1.0],
    'regressor__max_features':['auto','sqrt']
}

In [70]:
columns_to_encode=['property_type','sector','balcony','agePossession','furnishing_type','luxury_category','floor_category']

# creating a column tranformer for preprocessing


preprocessor=ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),['bedRoom','bathroom','built_up_area','servant room','store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc',ce.TargetEncoder(),['sector'])
    ],
    remainder='passthrough'
)


In [72]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [73]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [74]:
search =GridSearchCV(pipeline,param_grid,cv=kfold,scoring='r2',n_jobs=-1,verbose=4)

In [75]:
search.fit(X,y_transformed)

Fitting 10 folds for each of 128 candidates, totalling 1280 fits


GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['bedRoom',
                                                                          'bathroom',
                                                                          'built_up_area',
                                                                          'servant '
                                                                          'room',
                                                                          'store '
                                                                          'room']),
                                                                        ('cat',
                                                                         OrdinalEncoder(),
                                                                         ['property_type',
                                                                          'sector',
                                                                          'balcony',
                                                                          'agePossession',
                                                                          'furnishing_type',...
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['agePossession']),
                                                                        ('target_enc',
                                                                         TargetEncoder(),
                                                                         ['sector'])])),
                                       ('regressor', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [None, 10, 20, 30],
                         'regressor__max_features': ['auto', 'sqrt'],
                         'regressor__max_samples': [0.1, 0.25, 0.5, 1.0],
                         'regressor__n_estimators': [50, 100, 200, 300]},
             scoring='r2', verbose=4)

In [76]:
final_pipe=search.best_estimator_


In [77]:
search.best_params_


{'regressor__max_depth': 30,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__n_estimators': 100}

In [78]:
search.best_score_

0.9027272204264382

In [96]:
# final_pipe.fit(X,y_transformed)

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

print(scores.mean())


y_pred = final_pipe.predict(X_test)

y_pred = np.expm1(y_pred)

print(mean_absolute_error(np.expm1(y_test),y_pred))

0.8914659157607071
0.17764766050961947


### Exporting the model 

In [90]:
columns_to_encode=['property_type','sector','balcony','agePossession','furnishing_type','luxury_category','floor_category']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [91]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500))
])

In [92]:
pipeline.fit(X,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sector',
                                                   'agePossession'])])),
                ('regressor', RandomForestRegressor(n_estimators=500))])

In [94]:

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

print(scores.mean())


y_pred = pipeline.predict(X_test)

y_pred = np.expm1(y_pred)

print(mean_absolute_error(np.expm1(y_test),y_pred))

0.8922184849153124
0.17537902882119477


In [100]:
!pip install pickle-mixin

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6017 sha256=ec33255948cdf4e2c460339c5b6cf4e6fd0b4c8d9b8eda3830a58b7d448c08e8
  Stored in directory: c:\users\sk531\appdata\local\pip\cache\wheels\fc\3c\04\2932798281e4075dc4eadd6c6c79ee66630ee5266ae32731a3
Successfully built pickle-mixin


In [103]:
import pickle 

with open('pipeline.pkl','wb')as file :
    pickle.dump(pipeline,file)

In [104]:
with open('df.pkl','wb')as file:
    pickle.dump(X,file)

In [105]:
X

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor
1,flat,sector 89,2.0,2.0,2,New Property,1226.0,1.0,0.0,unfurnished,Low,Mid Floor
2,flat,sohna road,2.0,2.0,1,New Property,1000.0,0.0,0.0,unfurnished,Low,High Floor
3,flat,sector 92,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,semifurnished,High,Mid Floor
4,flat,sector 102,2.0,2.0,1,Relatively New,582.0,0.0,1.0,unfurnished,High,Mid Floor
...,...,...,...,...,...,...,...,...,...,...,...,...
3549,flat,sector 84,2.0,2.0,1,Relatively New,532.0,0.0,0.0,unfurnished,Medium,Mid Floor
3550,house,sector 109,5.0,5.0,3+,Relatively New,6228.0,1.0,1.0,unfurnished,High,Low Floor
3551,flat,sector 2,1.0,1.0,1,Moderately Old,665.0,0.0,0.0,semifurnished,Medium,Mid Floor
3552,house,sector 43,5.0,6.0,3,Moderately Old,5490.0,1.0,1.0,unfurnished,Medium,Mid Floor


### Trying out the predictions 

In [106]:
X.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [108]:
X.iloc[0].values

array(['flat', 'sector 36', 3.0, 2.0, '2', 'New Property', 850.0, 0.0,
       0.0, 'unfurnished', 'Low', 'Low Floor'], dtype=object)

In [ ]:
data=[['house','sector 102',4,3,'3+','New Property',2750,0,0, 'unfurnished', 'Low', 'Low Floor']]
columns= ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']


# convert to df 

one_df=pd.DataFrame(data,columns=columns)
one_df

In [112]:
np.expm1(pipeline.predict(one_df))

array([3.15099567])

In [113]:
X.dtypes

property_type       object
sector              object
bedRoom            float64
bathroom           float64
balcony             object
agePossession       object
built_up_area      float64
servant room       float64
store room         float64
furnishing_type     object
luxury_category     object
floor_category      object
dtype: object

In [114]:
sorted(X['sector'].unique().tolist())

['dwarka expressway',
 'gwal pahari',
 'manesar',
 'sector 1',
 'sector 10',
 'sector 102',
 'sector 103',
 'sector 104',
 'sector 105',
 'sector 106',
 'sector 107',
 'sector 108',
 'sector 109',
 'sector 11',
 'sector 110',
 'sector 111',
 'sector 112',
 'sector 113',
 'sector 12',
 'sector 13',
 'sector 14',
 'sector 15',
 'sector 17',
 'sector 2',
 'sector 21',
 'sector 22',
 'sector 23',
 'sector 24',
 'sector 25',
 'sector 26',
 'sector 27',
 'sector 28',
 'sector 3',
 'sector 30',
 'sector 31',
 'sector 33',
 'sector 36',
 'sector 37',
 'sector 37d',
 'sector 38',
 'sector 39',
 'sector 4',
 'sector 40',
 'sector 41',
 'sector 43',
 'sector 45',
 'sector 46',
 'sector 47',
 'sector 48',
 'sector 49',
 'sector 5',
 'sector 50',
 'sector 51',
 'sector 52',
 'sector 53',
 'sector 54',
 'sector 55',
 'sector 56',
 'sector 57',
 'sector 58',
 'sector 59',
 'sector 6',
 'sector 60',
 'sector 61',
 'sector 62',
 'sector 63',
 'sector 63a',
 'sector 65',
 'sector 66',
 'sector 67',
 'se